In [1]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.1 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
google-cloud-aiplatform 1.54.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
google-cloud-bigquery-connection 1.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but y

In [2]:
import os
import textwrap
from pathlib import Path

from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")


def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))

In [3]:
# !mkdir data
# !gdown 1ee-BhQiH-S9a2IkHiFbJz9eX_SfcZ5m9 -O "data/meta-earnings.pdf"

## Document Parsing

In [4]:
from google.colab import userdata
LLAMA_PARSE_API_KEY = userdata.get("LLAMA_PARSE_API_KEY")

In [5]:
instruction = """The provided document is a delivery chalan or invoice.
This form provides detailed chalan or invoice information about the company's performance for a specific quarter.
It includes unaudited invoice statements, management discussion and analysis, and other relevant disclosures required by the SEC.
It contains many tables.
Try to be precise while answering the questions"""

parser = LlamaParse(
    api_key=userdata.get("LLAMA_PARSE_API_KEY"),
    result_type="markdown",
    parsing_instruction=instruction,
    max_timeout=5000,
)

llama_parse_documents = await parser.aload_data("doc121.pdf")

Started parsing the file under job_id cac11eca-2515-4cba-8b5b-61642ab893c5


In [6]:
# llama_parse_documents

In [7]:
parsed_doc = llama_parse_documents[0]
# parsed_doc

In [8]:
# parsed_doc.text

In [9]:
Markdown(parsed_doc.text)

# DELIVERY Challan (UNDER SECTION 31 OF CAST RULES)

|CoalBill No|ksloiRDzs0004344|
|---|---|
|Ebillan No|B472 %atiesue|
|DQ Number|310000io58|
|DQ Date Removal|03-062024|
|Date of Removal|09.57.57|
|State of Consignor|Uttar Pradesh4-06-2024|
|State Code of Consignor|09|

# Details of Receiver (Billed to)

|Name|HINDALCO INDUSTRIES|
|---|---|
|Address|RENU SAGAR SONEBHADRA|
|State Code|Uttar Pradesh09|
|Unique ID|opakachizotriL|

# Details of Consignee (Shipped to)

|Name|HINDALCO INDUSTRIES LIMITED|
|---|---|
|Address|RENU SAGAR|
|State Code|Uttar Pradesh09|
|Unique ID|opakachizotriL|

# Product Details

|Product Grade|HSN 27011990 GB|
|---|---|
|Truck No|Up64t4948|
|Date In|14/06/2024 Time 08:31:18|
|Date of Removal|14/06/2024 Time 09:57:37|
|Gross Weight|40.56 MT|
|Tare Weight|13.08 MT|
|Net Weight|27.48 MT|
|Due Quantity|96100.00 MT|
|Balance Quantity|88731.86 MT|
|Validity|22-07-2024|

# Particulars Value

|Coal|3479.00|9560292|
|---|---|---|
|High loading Charges (Rs. per MT)|0.00|2390.76|
|Transportation Charges|112.12|301.00|
|Evacuation Facility Charges|14.09|643.09|
|NMET Funds for UP State (Rs. per MT)|6.30|21.30|
|MP Cess for MP State (Rs. per MT)|0.00|0.00|
|Territorial Forest Fee (Rs. per MT)|0.00|0.00|
|CGST (2.5%)|205.28| |
|SGST (2.5%)|205.28| |
|IGST (4%)|863.10| |
|Total Invoice Value|132854.92| |

Declaration: Payment to be made as per terms of FSA auction to be revised @ 5% per terms of FSA auction. Pre-Receipted Authenticated

WBC PARA NIAN SISX
---
# Or Igina] ar Buyet /oup!cateT0r Transportes /Tc [plcatefor

|Coe|Ho|BNaosrd750004506|
|---|---|---|
|chalian %o|4595|BI0fMiaoDale|14-06-7074|
|GSTIK No| | | |
|D0humpor|01 Nci|Isquo|08:55:45|
| |3001407462|00OatoRaaava||0a:55:44|
|Dato0tRemova|14-067074|Tin|09|
|Sato0fConsIgnor|Uttar_Pradesh|Stato Coda0fCons [anor|

# Dotails ot RoceIver (0|Mad to)

|LIMITEO ReNusaGAR| | | |
|---|---|---|---|
|Nanc|HIndALCO INDUSTRIES|Eo RENUSAGAR| |
|Addror;|RENUSAGAR|Addrcss|REMUSAGAR~onebhadra|
|Stato|UTTAR PRAOESH|State|UtTaR PradEsH|
|GSTIN UniqueId|stato Code|OPAAACHIZOIRIZL|09|
|GSTIN UniqueId|State Code|OJAAACHIZOIRIZL|09|

|Product|Grade|Bill forsupplyof27011990G9|Coa 250 Hmthrough Roadason date 14-06-2024|
|---|---|---|---|
|Truck No|Up649t761s| | |
|Date in|14/06/2024Time 08:16.39|Dateof removal 14/06/2024 Time -08.55:44| |
|Oross Wcight|43 21 HTBalance_QuantityTare Weight: 1233619.5975THTNet Weight:Validity20 46Hf| | |
|DoQuentity|63850.00_MT.| |29-06-2024|
|Particulars|Coa Va lue|3441.00|104812.86|
|A.Sizing Char gesHigh Capacity Loading Charges 2(56 Rs. per MT)| | |T705.760020|
|Transportation Charges| |112-12| |
|Evacuation Facfty Charges 2(60 Rs pet MT)| | | |
|Roya)ty|44&0f| | |
|OhFFunds|8030f| | |
|NMETFundsCess for UP state of| | | |
|MPGATSVA for MP State|Ssada Rs_per| | |
|Forest Transit Fee 0&of 0)|00| | |
|Terminal Tax 2{05-00perHT )| | |5287_86|
|ReimbursementofOMF|4533| | |
|ReimbursenentofNMET|T05 .| | |
|RembursementofHPGATSVAFees|F756 .| | |
|Reimburscment ofof_Ato_Q)Transit| | | |
|Sub Total Sum| | |22518.4|
|CQSTof 2E25}(RS| | |032|
|SGSTof 0(2.5 % (Rs| | |3062.96|
|IGSTof PfO*(Rs :| | |0700|
|Tota] E umof to U| | |128484 . 41|
|GST Compensation cesb/C E.CessOrtv Shousina {{ghtordnared Thenty EIghtRs, and Thirty Fhree Paisa Only4140828 >3e( 400 per _MD)| | |T12184300|
|NetAmount (In Figures one Lakh Forty ThousandTota| | | |
|2. Biils to be raiseda5 perto bemadea3pertecmsof_ESA/E_auctionterms of FSATE auction|7nab 641924| | |
|BC|PARTY sG|SALES MANAGER| |
---
# Original for Buyer [Duplicate for Transporter/Triplicate for Assessee

|CC E&iLE No|BNFOSRD250064575|
|---|---|
|Challan No of NCL|09AABCN4884HLZ4 4563|
|Bill Date|14-06-2024|
|DQ Number|3001402462|
|DO Date Issue|02-05-2024 07.53.44|
|Date of Removal|14-06-2024 07.53.42|
|State of Consignor|Uttar Pradesh State Code of Consignor 09|
|Financial Year 2024-25| |

|Details of Receiver (Billed to)|Details of Consignee (Shipped to)|
|---|---|
|EXISTED RENUSAGAR|HINDALCO INDUSTRIES|
|Address: RENUSAGAR SONEBHADRA|Address: RENUSAGAR-SONEBHADRA|
|State: Uttar Pradesh State Code: 09|State: Uttar Pradesh|
|GSTIN: QoaaaCHIZO1R1ZI 09|GSTIN: QoaaaCHIZO1R1ZI 09|

|Product & Grade|#SBi! for supply of Coal through Road 35Ondate 14-06-2024 27011990-G9 7250 MT|
|---|---|
|Truck No|Date: 44/5/2024 Time: 06.28-08 Date: 44/06/2024|
|Quantity|43.77 MT Balance Quantity Tare Weight: 1333962.1125 MT Net Validity Neight: 30 297067202452 MT|
|Particulars|Size Charges Coal Value: 256 RS per IE 3441.00 Total: 50.32|
|Transportation Charges|112.12|
|Addition Eaclty Charges|00.00|
|Royalties|3421.90 1831.20|
|ONHET Funds for UP State Df|2 (14(30 Zof 0000|
|SSAD Acess|0T 00|
|KM pGatsya Forest Transit Fee|Rs per JO|
|M.Terninal Tax Reimbursement of Royalty|0f 5298|
|Reimbursement of HPGATSYANRET|1532|
|Reimbursement of SubJota Fansil Fees|4892.24 105|
|Cost of Sum Q|1B964|
|SGST of|25 RS 122752.74|
|IGST of|2(2.5 Rs 306899|
|Total (Sum of A)|306899|
|E-Cess|2(400 per MT 728737.50030|
|Ret Amount (Figures Total Invoice Value (In Words)|42208.00 AFQS 73|

Declaration: Please Rechentzc Bills to be raised as Per terms. Payment to be made as per terms of ESA/Eauction. Authenticated Part 31 Sales Manager
---
# Oriqinel for Buyer/Duplicate for NORTHERN COALFIELDS LIMITED Transporter/Triplicate of Assessee

|Delivery Challan|No|BNAOspd250004972|
|---|---|---|
|Challen No of Kci|4570|8i 41Oato|
|Date Issued|07:44:54|14-06-2024|
|DO Number|3001402462|02-05-2024|
|State of Consignor|Uttar Pradesh|State Code of Consignor|07|

# Details of Receiver (Billed to)

|Limited Renusagar|HINDALCO INDUSTRIES|
|---|---|
|Address|RENUSAGAR|
|State|Uttar Pradesh|
|GSIN (Unique ID)|O9aaaCHi?O1RIZL|087 iN/UniqueId|O2AAACHIZO[RIZE|

# Product Details

|Product|Grade|#sBi! !fo5,supply0fCoa] through Road35ondate 14-06-202427011990 G9|250 Hh|
|---|---|---|---|
|Truck No|Upg44t47s|Date|14/06/2024|
|Gross Weight|44.91 MT|Tare Weight|334051.4150 MtMT|
|Net Weight|63850.00 MT|Validity|31-41 MT|

# Particulars

|A. Sizing Charges|Coal Value|3441 Co|10368E4752|
|---|---|---|---|
|High Capacity Loading Charges|0.00|352199| |
|Transportation Charges|0.00| | |
|Evacuation Facility Charges|112.12| | |
|Royalty|4884.60| | |
|DHF Funds|0.00| | |
|NHET Funds for UP State|0.00| | |
|Saad Access|0.00| | |
|Forest Transit Fee|0.00| | |
|HPGATSYA for KP State|0.00| | |
|Jax|0.00| | |
|Reimbursement of Royalty|157.05| | |
|Reimbursement of OHF|1635.83| | |
|Reimbursement of NMET|5452.78| | |
|Reimbursement of MpGatsva|109.06| | |
|Reimbursement of Transit Fees|1790.37| | |
|CGST Total|126339.5€| | |
|SGST|3158.49| | |
|IGST|3158.49| | |
|GST Compensation Cess/C.E.Cess|132491.690.00| | |
|Net Amount (In Figures)|12564.00| | |
|Total Invoice Value (In Words)|145220.54| | |

One Lakh Forty-Five Thousand Two Hundred Twenty Rs. and Fifty-Four Paisa Only

Declaration: Bills to be raised as per terms of FSAYE auction. Payment to be made as per terms of FSA/E auction. Received & authenticated by: SACES MANAGER

In [10]:
# Markdown(parsed_doc.text[:4096])

In [11]:
document_path = Path("parsed_document.md")
with document_path.open("a") as f:
    f.write(parsed_doc.text)

## Vector Embeddings

In [12]:
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
# len(docs)

In [14]:
print(docs[0].page_content)

DELIVERY Challan (UNDER SECTION 31 OF CAST RULES)

CoalBill No ksloiRDzs0004344 Ebillan No B472 %atiesue DQ Number 310000io58 DQ Date Removal 03-062024 Date of Removal 09.57.57 State of Consignor Uttar Pradesh4-06-2024 State Code of Consignor 09

Details of Receiver (Billed to)

Name HINDALCO INDUSTRIES Address RENU SAGAR SONEBHADRA State Code Uttar Pradesh09 Unique ID opakachizotriL

Details of Consignee (Shipped to)

Name HINDALCO INDUSTRIES LIMITED Address RENU SAGAR State Code Uttar Pradesh09 Unique ID opakachizotriL

Product Details

Product Grade HSN 27011990 GB Truck No Up64t4948 Date In 14/06/2024 Time 08:31:18 Date of Removal 14/06/2024 Time 09:57:37 Gross Weight 40.56 MT Tare Weight 13.08 MT Net Weight 27.48 MT Due Quantity 96100.00 MT Balance Quantity 88731.86 MT Validity 22-07-2024

Particulars Value

Coal 3479.00 9560292 High loading Charges (Rs. per MT) 0.00 2390.76 Transportation Charges 112.12 301.00 Evacuation Facility Charges 14.09 643.09 NMET Funds for UP State (Rs. 

In [15]:
HF_TOKEN = userdata.get("HF_TOKEN")

In [16]:
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

In [17]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    # location=":memory:",
    path="./db",
    collection_name="document_embeddings",
)

In [18]:
# %%time
# query = "Summarize the document. Share response in JSON format."
# similar_docs = qdrant.similarity_search_with_score(query)

In [19]:
# for doc, score in similar_docs:
#     print(f"text: {doc.page_content[:256]}\n")
#     print(f"score: {score}")
#     print("-" * 80)
#     print()

In [20]:
# %%time
retriever = qdrant.as_retriever(search_kwargs={"k": 5})
# retrieved_docs = retriever.invoke(query)

In [21]:
# for doc in retrieved_docs:
#     print(f"id: {doc.metadata['_id']}\n")
#     print(f"text: {doc.page_content[:256]}\n")
#     print("-" * 80)
#     print()

## Reranking

In [22]:
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
compression_retriever = ContextualCompressionRetriever(
    base_compressor = compressor, base_retriever = retriever
)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:00<00:00, 86.2MiB/s]


In [23]:
# %%time
# reranked_docs = compression_retriever.invoke(query)
# len(reranked_docs)

In [24]:
# for doc in reranked_docs:
#     print(f"id: {doc.metadata['_id']}\n")
#     print(f"text: {doc.page_content[:256]}\n")
#     print(f"score: {doc.metadata['relevance_score']}")
#     print("-" * 80)
#     print()

## Q&A Over Document

In [25]:
llm = ChatGroq(temperature = 0,
               model_name = "llama3-70b-8192")

In [26]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(template = prompt_template,
                        input_variables = ["context", "question"])

In [27]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = compression_retriever,
    return_source_documents = True,
    chain_type_kwargs = {"prompt" : prompt, "verbose" : True},
)

In [28]:
%%time
query = "Summarize the document. Share response in JSON format."
response = qa.invoke(str(query))

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Product & Grade #SBi! for supply of Coal through Road 35Ondate 14-06-2024 27011990-G9 7250 MT Truck No Date: 44/5/2024 Time: 06.28-08 Date: 44/06/2024 Quantity 43.77 MT Balance Quantity Tare Weight: 1333962.1125 MT Net Validity Neight: 30 297067202452 MT Particulars Size Charges Coal Value: 256 RS per IE 3441.00 Total: 50.32 Transportation Charges 112.12 Addition Eaclty Charges 00.00 Royalties 3421.90 1831.20 ONHET Funds for UP State Df 2 (14(30 Zof 0000 SSAD Acess 0T 00 KM pGatsya Forest Transit Fee Rs per JO M.Terninal Tax Reimbursement of Royalty 0f 5298 Reimbursement of HPGATSYANRET 1532 Reimbursement of SubJota Fansil Fees 4892.24 105 Cost of Sum Q 1B964 SGST of 25 RS 122752.74 I

In [29]:
# print_response(response)

In [30]:
from IPython.display import Markdown
Markdown(response["result"])

Here is a summary of the document in JSON format:

```
{
  "Document Type": "Coal Bill",
  "Product Details": {
    "Product Grade": "#SBi!",
    "HSN": "27011990",
    "Quantity": "250 MT",
    "Truck No": "Up64t4948",
    "Date": "14-06-2024",
    "Time": "08:31:18"
  },
  "Consignor Details": {
    "Name": "NORTHERN COALFIELDS LIMITED",
    "Address": "Uttar Pradesh",
    "State Code": "09"
  },
  "Consignee Details": {
    "Name": "HINDALCO INDUSTRIES LIMITED",
    "Address": "RENUSAGAR, SONEBHADRA",
    "State Code": "09",
    "GSTIN": "09aaaCHIZO1RIZL"
  },
  "Invoice Details": {
    "Invoice Value": "132854.92",
    "CGST": "205.28",
    "SGST": "205.28",
    "IGST": "863.10"
  }
}
```

Additional helpful information:

* The document appears to be a coal bill for the supply of coal through road transportation.
* The bill is raised by NORTHERN COALFIELDS LIMITED and is addressed to HINDALCO INDUSTRIES LIMITED.
* The invoice value is ₹1,328,549.92, with CGST, SGST, and IGST components.
* The document includes details of the product, consignor, consignee, and invoice values.

In [31]:
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=compression_retriever,
#     return_source_documents=True,
#     chain_type_kwargs={"prompt": prompt, "verbose": False},
# )

In [32]:
# %%time
# response = qa.invoke("What is the revenue for 2024 and % change?")

In [33]:
# Markdown(response["result"])

In [34]:
# %%time
# response = qa.invoke("What is the revenue for 2023?")

In [35]:
# print_response(response)

In [36]:
# Markdown(response["result"])

In [37]:
# %%time
# response = qa.invoke(
#     "How much is the revenue minus the costs and expenses for 2024? Calculate the answer"
# )

In [38]:
# print_response(response)

In [39]:
# Markdown(response["result"])

In [40]:
# %%time
# response = qa.invoke(
#     "How much is the revenue minus the costs and expenses for 2023? Calculate the answer"
# )

In [41]:
# print_response(response)

In [42]:
# Markdown(response["result"])

In [43]:
# %%time
# response = qa.invoke("What is the expected revenue for the second quarter of 2024?")

In [44]:
# Markdown(response["result"])

In [45]:
# %%time
# response = qa.invoke("What is the overall outlook of Q1 2024?")

In [46]:
# print_response(response)

In [47]:
# Markdown(response["result"])

## References

- [Meta Reports First Quarter 2024 Results](https://s21.q4cdn.com/399680738/files/doc_financials/2024/q1/Meta-03-31-2024-Exhibit-99-1_FINAL.pdf)